<a href="https://colab.research.google.com/github/adadoun/inventoryPlanningRecommendation/blob/main/EDAStepSizeOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sales Data Analysis

This notebook contains an analysis of the sales data in order to find out the best step size to use for sales prediction. We need to decide of the sales prediction, will be done by day, by week or by month

Finding the Optimal Step Size

1. Exploratory Data Analysis (EDA):
* Plot sales at different granularities
* Look for clear patterns or noise


2. Autocorrelation Analysis: Use autocorrelation and partial autocorrelation plots to identify significant lags

In [ ]:
# Import required libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import hashlib
from statsmodels.tsa.stattools import acf
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the dataset
df = pd.read_csv('drive/MyDrive/Collab_DATA/PolarData/sales_data_csv.csv')

## Data Transformation

In [ ]:
# Function to hash SKU
def hash_sku(sku):
   return hashlib.md5(str(sku).encode()).hexdigest()[:8]
# Apply hash function to SKU column
df['SKU'] = df['SKU'].apply(hash_sku)

# Transform date to datetime column
df['DATE'] = pd.to_datetime(df['DATE'])
df = df[df.DATE > "2020-01-01"]

# Create different Date granularities
daily = df.set_index('DATE').groupby(['DATE', 'SKU'])['QUANTITY_SOLD'].sum().unstack()
weekly = df.set_index('DATE').groupby([pd.Grouper(freq='W'), 'SKU'])['QUANTITY_SOLD'].sum().unstack()
monthly = df.set_index('DATE').groupby([pd.Grouper(freq='M'), 'SKU'])['QUANTITY_SOLD'].sum().unstack()

### 1. Sales distribution for different granularities (by SKU)

In [ ]:
def plot_sales_at_different_granularities(daily: pd.DataFrame, weekly: pd.DataFrame, monthly: pd.DataFrame) -> None:
    """
    Create a multi-panel plot showing sales trends at daily, weekly, and monthly granularities.

    This function takes pre-aggregated sales data at different time granularities and
    creates a three-panel plot to visualize sales trends over time at these different scales.

    Args:
        daily (pd.DataFrame): DataFrame with daily sales data. Index should be datetime, first column should be sales.
        weekly (pd.DataFrame): DataFrame with weekly sales data. Index should be datetime, first column should be sales.
        monthly (pd.DataFrame): DataFrame with monthly sales data. Index should be datetime, first column should be sales.

    Returns:
        None: Displays the plot using plotly's fig.show().
    """

    # Create subplots: one row for each time granularity
    fig = make_subplots(rows=3, cols=1, subplot_titles=('Daily', 'Weekly', 'Monthly'))

    # Add daily sales trace
    fig.add_trace(
        go.Scatter(x=daily.index, y=daily.iloc[:, 1], mode='lines', name='Daily'),
        row=1, col=1
    )

    # Add weekly sales trace
    fig.add_trace(
        go.Scatter(x=weekly.index, y=weekly.iloc[:, 1], mode='lines', name='Weekly'),
        row=2, col=1
    )

    # Add monthly sales trace
    fig.add_trace(
        go.Scatter(x=monthly.index, y=monthly.iloc[:, 1], mode='lines', name='Monthly'),
        row=3, col=1
    )

    # Update layout
    fig.update_layout(height=900, title_text="Sales at Different Time Granularities")

    # Display the plot
    fig.show()

plot_sales_at_different_granularities(daily, weekly, monthly)


## 2. Missing periods analysis

In [ ]:
def analyze_missing_periods(df: pd.DataFrame) -> dict:
    """
    Analyze missing time periods (days, weeks, months) for each SKU in the dataset.

    This function identifies the full date range of the dataset and then, for each SKU,
    calculates how many days, weeks, and months are missing from its data.

    Args:
        df (pd.DataFrame): A DataFrame containing sales data with 'DATE' and 'SKU' columns.

    Returns:
        dict: A nested dictionary with SKUs as keys and dictionaries of missing period counts as values.
    """
    # Ensure DATE is datetime
    df['DATE'] = pd.to_datetime(df['DATE'])

    # Get the full date range
    start_date = df['DATE'].min()
    end_date = df['DATE'].max()
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')

    results = {}

    for sku in df['SKU'].unique():
        sku_data = df[df['SKU'] == sku]

        # Daily analysis
        days_present = sku_data['DATE'].dt.date.unique()
        missing_days = len(full_date_range) - len(days_present)

        # Weekly analysis
        weeks_present = sku_data['DATE'].dt.to_period('W').unique()
        all_weeks = full_date_range.to_period('W').unique()
        missing_weeks = len(all_weeks) - len(weeks_present)

        # Monthly analysis
        months_present = sku_data['DATE'].dt.to_period('M').unique()
        all_months = full_date_range.to_period('M').unique()
        missing_months = len(all_months) - len(months_present)

        results[sku] = {
            'missing_days': missing_days,
            'missing_weeks': missing_weeks,
            'missing_months': missing_months,
            'total_days': len(full_date_range),
            'total_weeks': len(all_weeks),
            'total_months': len(all_months)
        }

    return results

def plot_missing_periods(results: dict) -> None:
    """
    Create violin plots showing the distribution of missing time periods for all SKUs.

    This function takes the results from analyze_missing_periods and creates three violin plots,
    one each for missing days, weeks, and months across all SKUs.

    Args:
        results (dict): The nested dictionary returned by analyze_missing_periods.

    Returns:
        None: Displays the plot using plotly's fig.show().
    """
    skus = list(results.keys())
    missing_days = [results[sku]['missing_days'] for sku in skus]
    missing_weeks = [results[sku]['missing_weeks'] for sku in skus]
    missing_months = [results[sku]['missing_months'] for sku in skus]

    fig = make_subplots(rows=1, cols=3,
                        subplot_titles=("Missing Days", "Missing Weeks", "Missing Months"),
                        shared_yaxes=True)

    fig.add_trace(go.Violin(y=missing_days, name="Days", box_visible=True, meanline_visible=True), row=1, col=1)
    fig.add_trace(go.Violin(y=missing_weeks, name="Weeks", box_visible=True, meanline_visible=True), row=1, col=2)
    fig.add_trace(go.Violin(y=missing_months, name="Months", box_visible=True, meanline_visible=True), row=1, col=3)

    fig.update_layout(
        title_text="Distribution of Missing Time Periods per SKU",
        height=600,
        width=1000
    )

    fig.show()

# Main execution
results = analyze_missing_periods(df)

# Print summary statistics
print("Summary Statistics:")
for period in ['days', 'weeks', 'months']:
    missing_counts = [results[sku][f'missing_{period}'] for sku in results]
    total = results[list(results.keys())[0]][f'total_{period}']
    print(f"\nMissing {period.capitalize()}:")
    print(f"  Average: {sum(missing_counts) / len(missing_counts):.2f}")
    print(f"  Median: {sorted(missing_counts)[len(missing_counts)//2]}")
    print(f"  Min: {min(missing_counts)}")
    print(f"  Max: {max(missing_counts)}")
    print(f"  Total {period}: {total}")

# Generate and display the violin plots
plot_missing_periods(results)

Summary Statistics:

Missing Days:
  Average: 1379.04
  Median: 1531
  Min: 1
  Max: 1603
  Total days: 1604

Missing Weeks:
  Average: 161.73
  Median: 196
  Min: 0
  Max: 229
  Total weeks: 230

Missing Months:
  Average: 32.21
  Median: 40
  Min: 0
  Max: 52
  Total months: 53


In [ ]:
def compute_lag_correlations(df: pd.DataFrame, sku: str, max_lags: int = 30) -> tuple:
    """
    Compute autocorrelations for daily, weekly, and monthly sales data for a given SKU.

    Args:
        df (pd.DataFrame): DataFrame containing sales data with 'DATE', 'SKU', and 'QUANTITY_SOLD' columns.
        sku (str): The SKU to analyze.
        max_lags (int, optional): Maximum number of lags to compute. Defaults to 30.

    Returns:
        tuple: Three numpy arrays containing autocorrelations for daily, weekly, and monthly data.
    """
    # Resample to daily, weekly, and monthly frequencies
    daily = df[df['SKU'] == sku].set_index('DATE')['QUANTITY_SOLD'].resample('D').sum()
    weekly = df[df['SKU'] == sku].set_index('DATE')['QUANTITY_SOLD'].resample('W').sum()
    monthly = df[df['SKU'] == sku].set_index('DATE')['QUANTITY_SOLD'].resample('M').sum()

    # Compute autocorrelations
    daily_acf = acf(daily.dropna(), nlags=max_lags)
    weekly_acf = acf(weekly.dropna(), nlags=max_lags)
    monthly_acf = acf(monthly.dropna(), nlags=max_lags)

    return daily_acf, weekly_acf, monthly_acf

def plot_lag_correlations(daily_acf: np.array, weekly_acf: np.array, monthly_acf: np.array, sku: str) -> None:
    """
    Plot lag correlations for daily, weekly, and monthly data.

    Args:
        daily_acf (np.array): Autocorrelations for daily data.
        weekly_acf (np.array): Autocorrelations for weekly data.
        monthly_acf (np.array): Autocorrelations for monthly data.
        sku (str): The SKU being analyzed.

    Returns:
        None: Displays the plot using plotly's fig.show().
    """
    fig = make_subplots(rows=3, cols=1,
                        subplot_titles=("Daily Lag Correlations", "Weekly Lag Correlations", "Monthly Lag Correlations"),
                        shared_xaxes=True)

    lags = list(range(len(daily_acf)))

    fig.add_trace(go.Bar(x=lags, y=daily_acf, name="Daily"), row=1, col=1)
    fig.add_trace(go.Bar(x=lags, y=weekly_acf, name="Weekly"), row=2, col=1)
    fig.add_trace(go.Bar(x=lags, y=monthly_acf, name="Monthly"), row=3, col=1)

    fig.update_layout(
        title_text=f"Lag Correlations for SKU: {sku}",
        height=900,
        width=1000,
        showlegend=False
    )

    fig.update_yaxes(title_text="Correlation", range=[-1, 1])
    fig.update_xaxes(title_text="Lag")

    fig.show()

# Main execution
sku_to_analyze = df['SKU'].unique()[4]

# Compute lag correlations
daily_acf, weekly_acf, monthly_acf = compute_lag_correlations(df, sku_to_analyze)

# Plot lag correlations
plot_lag_correlations(daily_acf, weekly_acf, monthly_acf, sku_to_analyze)

# Print top 5 lag correlations for each granularity
print("Top 5 Daily Lag Correlations:")
print(pd.Series(daily_acf[1:]).nlargest(5))  # Exclude lag 0 (always 1)

print("\nTop 5 Weekly Lag Correlations:")
print(pd.Series(weekly_acf[1:]).nlargest(5))  # Exclude lag 0 (always 1)

print("\nTop 5 Monthly Lag Correlations:")
print(pd.Series(monthly_acf[1:]).nlargest(5))  # Exclude lag 0 (always 1)

# Create scatter plots for the top lag in each granularity
top_daily_lag = pd.Series(daily_acf[1:]).nlargest(1).index[0] + 1
top_weekly_lag = pd.Series(weekly_acf[1:]).nlargest(1).index[0] + 1
top_monthly_lag = pd.Series(monthly_acf[1:]).nlargest(1).index[0] + 1

Top 5 Daily Lag Correlations:
16    0.102622
27    0.077656
2     0.070813
0     0.043841
1     0.043597
dtype: float64

Top 5 Weekly Lag Correlations:
24    0.230894
21    0.135631
2     0.069280
18    0.047390
5     0.029010
dtype: float64

Top 5 Monthly Lag Correlations:
10    0.260714
4     0.088393
3     0.035714
5     0.034821
11   -0.017857
dtype: float64


In [ ]:
def compute_general_lag_stats(df: pd.DataFrame, top_skus: list, max_lags: int = 30) -> pd.DataFrame:
    """
    Compute general lag correlation statistics for top SKUs across different time periods.
    """
    periods = {'daily': 'D', 'weekly': 'W', 'monthly': 'M'}
    stats = []

    for period_name, period_code in periods.items():
        all_acf_values = []

        for sku in top_skus:
            sku_data = df[df['SKU'] == sku].set_index('DATE')['QUANTITY_SOLD']
            resampled_data = sku_data.resample(period_code).sum()
            if len(resampled_data) > max_lags + 1:
                acf_values = acf(resampled_data.dropna(), nlags=max_lags, fft=False)[1:]
                all_acf_values.extend(acf_values)

        if all_acf_values:
            all_acf_values = np.array(all_acf_values)
            n = len(all_acf_values) + 1
            sig_level = 1.96 / np.sqrt(n)
            stats.append({
                'Period': period_name,
                'Mean': np.mean(all_acf_values),
                'Median': np.median(all_acf_values),
                'Std': np.std(all_acf_values),
                'Abs_Mean': np.mean(np.abs(all_acf_values)),
                'Pos_Frac': np.mean(all_acf_values > 0),
                'Neg_Frac': np.mean(all_acf_values < 0),
                'Sig_Lags_Frac': np.mean(np.abs(all_acf_values) > sig_level),
                'Top_5_Avg_Corr': np.mean(np.sort(np.abs(all_acf_values))[-5:])
            })

    return pd.DataFrame(stats)

# Get top 10 SKUs
top_10_skus = df.groupby('SKU')['QUANTITY_SOLD'].sum().nlargest(500).index.tolist()

# Compute general statistics
stats_df = compute_general_lag_stats(df, top_10_skus)

# Display statistics
print("General Lag Correlation Statistics for Top 10 SKUs Combined:")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(stats_df)

General Lag Correlation Statistics for Top 10 SKUs Combined:
    Period      Mean    Median       Std  Abs_Mean  Pos_Frac  Neg_Frac  Sig_Lags_Frac  \
0    daily  0.217431  0.177610  0.184912  0.221742  0.935475  0.064525       0.919344   
1   weekly  0.204995  0.171094  0.234849  0.237265  0.780791  0.219209       0.925706   
2  monthly  0.047928 -0.015083  0.254238  0.194569  0.467372  0.532628       0.927072   

   Top_5_Avg_Corr  
0        0.910827  
1        0.920366  
2        0.910952  
